In [3]:
from google.colab import drive
drive.mount('/content/drive')

!unzip /content/drive/MyDrive/IMSAI/image.zip -d /content/sample_data/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/IMSAI/image.zip
replace /content/sample_data/image/o_1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from sklearn.model_selection import KFold

In [5]:
dir_ = Path('/content/sample_data/image')

image_files = list(dir_.glob('*.png'))

In [7]:
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array


# 데이터와 레이블 리스트 초기화
data = []
labels = []

# 이미지와 레이블 로드 및 변환
for image_file in image_files:
    image = load_img(image_file, target_size=(300, 300), color_mode='grayscale')
    image = img_to_array(image)
    data.append(image)

    # 레이블 추출 (파일 이름의 첫 글자가 레이블)
    label = str(image_file.name)[0]
    labels.append(label)

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)

# 훈련:테스트 = 8:2
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# MLP
model = Sequential()
model.add(Flatten(input_shape=(300, 300, 1)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')


Epoch 1/20
5/5 [==============================] - 4s 601ms/step - loss: 23.9269 - accuracy: 0.4187 - val_loss: 18.1040 - val_accuracy: 0.4500
Epoch 2/20
5/5 [==============================] - 4s 785ms/step - loss: 12.7143 - accuracy: 0.4875 - val_loss: 4.4116 - val_accuracy: 0.5500
Epoch 3/20
5/5 [==============================] - 4s 712ms/step - loss: 7.2886 - accuracy: 0.5250 - val_loss: 12.2927 - val_accuracy: 0.5500
Epoch 4/20
5/5 [==============================] - 2s 411ms/step - loss: 14.7292 - accuracy: 0.5312 - val_loss: 0.7833 - val_accuracy: 0.5500
Epoch 5/20
5/5 [==============================] - 2s 408ms/step - loss: 6.5096 - accuracy: 0.5625 - val_loss: 3.9035 - val_accuracy: 0.4500
Epoch 6/20
5/5 [==============================] - 2s 412ms/step - loss: 8.6113 - accuracy: 0.5188 - val_loss: 13.8452 - val_accuracy: 0.4500
Epoch 7/20
5/5 [==============================] - 2s 436ms/step - loss: 9.2969 - accuracy: 0.4625 - val_loss: 7.2088 - val_accuracy: 0.5500
Epoch 8/20
5/5

In [8]:
# K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
accuracies = []

for train_index, test_index in kf.split(data):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # MLP
    model = Sequential()
    model.add(Flatten(input_shape=(300, 300, 1)))  # 흑백 이미지이므로 (300, 300, 1)
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    print(f'Training fold {fold_no}...')
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Fold {fold_no} - Test accuracy: {accuracy}')
    accuracies.append(accuracy)

    fold_no += 1

#결과
print(f'Average accuracy: {np.mean(accuracies)}')


Training fold 1...
Epoch 1/20
5/5 [==============================] - 4s 505ms/step - loss: 22.3311 - accuracy: 0.5250 - val_loss: 3.3695 - val_accuracy: 0.4500
Epoch 2/20
5/5 [==============================] - 2s 434ms/step - loss: 22.4031 - accuracy: 0.4000 - val_loss: 13.6457 - val_accuracy: 0.5500
Epoch 3/20
5/5 [==============================] - 2s 405ms/step - loss: 11.1206 - accuracy: 0.4437 - val_loss: 0.7765 - val_accuracy: 0.5750
Epoch 4/20
5/5 [==============================] - 3s 559ms/step - loss: 11.5106 - accuracy: 0.4375 - val_loss: 5.5270 - val_accuracy: 0.5500
Epoch 5/20
5/5 [==============================] - 3s 692ms/step - loss: 9.0058 - accuracy: 0.5063 - val_loss: 3.5467 - val_accuracy: 0.5500
Epoch 6/20
5/5 [==============================] - 2s 410ms/step - loss: 1.3343 - accuracy: 0.6750 - val_loss: 2.8694 - val_accuracy: 0.5500
Epoch 7/20
5/5 [==============================] - 2s 432ms/step - loss: 1.7649 - accuracy: 0.6250 - val_loss: 2.1595 - val_accuracy: 0.4